# Phase 6: Dashboard Data Merge

## Aadhaar Pulse - Final Backend-Ready Dataset

**Objective:** Merge all Phase 3-5 outputs into a single, unified dataset for backend APIs and frontend dashboards.

### Input Files:
| File | Source Phase | Fields |
|------|--------------|--------|
| `migration_intensity.csv` | Phase 3 | `migration_score`, `migration_category` |
| `peri_urban_labels.csv` | Phase 4 | `peri_urban_label` |
| `digital_exclusion_risk.csv` | Phase 5 | `digital_exclusion_score`, `risk_level` |

### Output Files:
| File | Format | Key Convention |
|------|--------|----------------|
| `aadhaar_pulse_dashboard.csv` | CSV | `snake_case` |
| `aadhaar_pulse_dashboard.json` | JSON | `camelCase` |

### Rules:
- ❌ NO new logic or recomputation
- ❌ NO value modifications
- ✅ Merge on `district_id` + `month`
- ✅ Preserve all original values

## 1. Import Libraries and Configure Paths

In [1]:
"""
Phase 6: Dashboard Data Merge
-----------------------------
This notebook merges all intelligence outputs into a single backend-ready dataset.

NO new logic is applied - only merging and formatting.
"""
import pandas as pd
import json
from pathlib import Path

# =============================================================================
# PATH CONFIGURATION
# =============================================================================
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()

# Input paths (from previous phases)
MIGRATION_PATH = PROJECT_ROOT / 'data' / 'final' / 'migration_intensity.csv'
PERI_URBAN_PATH = PROJECT_ROOT / 'data' / 'final' / 'peri_urban_labels.csv'
DIGITAL_EXCLUSION_PATH = PROJECT_ROOT / 'data' / 'final' / 'digital_exclusion_risk.csv'

# Output paths
OUTPUT_DIR = PROJECT_ROOT / 'data' / 'api_ready'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

CSV_OUTPUT_PATH = OUTPUT_DIR / 'aadhaar_pulse_dashboard.csv'
JSON_OUTPUT_PATH = OUTPUT_DIR / 'aadhaar_pulse_dashboard.json'

print("✅ Libraries imported and paths configured")
print(f"\n📁 Input files:")
print(f"   Migration:         {MIGRATION_PATH}")
print(f"   Peri-urban:        {PERI_URBAN_PATH}")
print(f"   Digital exclusion: {DIGITAL_EXCLUSION_PATH}")
print(f"\n📁 Output directory: {OUTPUT_DIR}")

✅ Libraries imported and paths configured

📁 Input files:
   Migration:         d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\final\migration_intensity.csv
   Peri-urban:        d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\final\peri_urban_labels.csv
   Digital exclusion: d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\final\digital_exclusion_risk.csv

📁 Output directory: d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\api_ready


## 2. Load Input Datasets

Load all three phase outputs and verify their structure.

In [2]:
"""
Load Input Datasets
-------------------
Load outputs from Phases 3, 4, and 5.
Verify row counts match before merging.
"""

# Load Phase 3: Migration Intensity
# Columns: district_id, month, migration_score, migration_category
migration_df = pd.read_csv(MIGRATION_PATH)
print(f"✅ Migration data loaded: {len(migration_df)} rows")
print(f"   Columns: {list(migration_df.columns)}")

# Load Phase 4: Peri-Urban Labels
# Columns: district_id, month, peri_urban_label, growth_confidence
peri_urban_df = pd.read_csv(PERI_URBAN_PATH)
print(f"\n✅ Peri-urban data loaded: {len(peri_urban_df)} rows")
print(f"   Columns: {list(peri_urban_df.columns)}")

# Load Phase 5: Digital Exclusion Risk
# Columns: district_id, month, digital_exclusion_score, risk_level
digital_df = pd.read_csv(DIGITAL_EXCLUSION_PATH)
print(f"\n✅ Digital exclusion data loaded: {len(digital_df)} rows")
print(f"   Columns: {list(digital_df.columns)}")

# Verify row count consistency
assert len(migration_df) == len(peri_urban_df) == len(digital_df), \
    "❌ Row count mismatch across input files!"
print(f"\n✅ Row count consistency verified: {len(migration_df)} rows each")

✅ Migration data loaded: 240 rows
   Columns: ['district_id', 'month', 'migration_score', 'migration_category']

✅ Peri-urban data loaded: 240 rows
   Columns: ['district_id', 'month', 'peri_urban_label', 'growth_confidence']

✅ Digital exclusion data loaded: 240 rows
   Columns: ['district_id', 'month', 'digital_exclusion_score', 'risk_level']

✅ Row count consistency verified: 240 rows each


## 3. Merge Datasets

Merge all three datasets on `district_id` + `month` keys.
Only select required columns for the final schema.

In [3]:
"""
Merge Datasets
--------------
Step 1: Start with migration data (has district_id, month, migration_score, migration_category)
Step 2: Merge peri-urban data (add peri_urban_label)
Step 3: Merge digital exclusion data (add digital_exclusion_score, risk_level)

Merge key: district_id + month
Join type: inner (all datasets should have identical keys)
"""

# Step 1: Start with migration data
# Keep: district_id, month, migration_score, migration_category
dashboard_df = migration_df[['district_id', 'month', 'migration_score', 'migration_category']].copy()
print(f"Step 1: Base migration data - {len(dashboard_df)} rows")

# Step 2: Merge peri-urban labels
# Keep only: peri_urban_label (drop growth_confidence - not in final schema)
peri_urban_subset = peri_urban_df[['district_id', 'month', 'peri_urban_label']]
dashboard_df = dashboard_df.merge(
    peri_urban_subset,
    on=['district_id', 'month'],
    how='inner'
)
print(f"Step 2: After peri-urban merge - {len(dashboard_df)} rows")

# Step 3: Merge digital exclusion data
# Keep: digital_exclusion_score, risk_level
digital_subset = digital_df[['district_id', 'month', 'digital_exclusion_score', 'risk_level']]
dashboard_df = dashboard_df.merge(
    digital_subset,
    on=['district_id', 'month'],
    how='inner'
)
print(f"Step 3: After digital exclusion merge - {len(dashboard_df)} rows")

# Verify no rows lost during merge
assert len(dashboard_df) == len(migration_df), \
    f"❌ Row loss during merge! Expected {len(migration_df)}, got {len(dashboard_df)}"
print(f"\n✅ Merge complete: {len(dashboard_df)} rows preserved")

Step 1: Base migration data - 240 rows
Step 2: After peri-urban merge - 240 rows
Step 3: After digital exclusion merge - 240 rows

✅ Merge complete: 240 rows preserved


## 4. Validate Final Schema and Data Quality

Ensure the merged dataset matches the exact required schema.

In [4]:
"""
Validate Final Schema and Data Quality
--------------------------------------
Check that the merged dataset conforms to the exact API schema.
"""

# =============================================================================
# FINAL SCHEMA DEFINITION (SOURCE OF TRUTH)
# =============================================================================
FINAL_COLUMNS = [
    'district_id',           # string - stable district identifier
    'month',                 # string - YYYY-MM format
    'migration_score',       # float - normalized migration intensity
    'migration_category',    # string - categorical label
    'peri_urban_label',      # string - urbanization classification
    'digital_exclusion_score',  # integer - 0-100 risk index
    'risk_level'             # string - Low/Medium/High
]

# Reorder columns to match final schema
dashboard_df = dashboard_df[FINAL_COLUMNS]

# Sort by district_id and month for consistent output
dashboard_df = dashboard_df.sort_values(['district_id', 'month']).reset_index(drop=True)

print("📋 Final Schema Validation:")
print(f"   Columns: {list(dashboard_df.columns)}")
print(f"   Expected: {FINAL_COLUMNS}")
assert list(dashboard_df.columns) == FINAL_COLUMNS, "❌ Schema mismatch!"
print("   ✅ Schema matches exactly")

# Check for missing values
null_counts = dashboard_df.isnull().sum()
if null_counts.sum() > 0:
    print(f"\n❌ Missing values found:")
    print(null_counts[null_counts > 0])
else:
    print(f"\n✅ No missing values")

# Check data types
print(f"\n📊 Data Types:")
for col in FINAL_COLUMNS:
    print(f"   {col}: {dashboard_df[col].dtype}")

# Preview merged data
print(f"\n📄 Sample merged data:")
dashboard_df.head(10)

📋 Final Schema Validation:
   Columns: ['district_id', 'month', 'migration_score', 'migration_category', 'peri_urban_label', 'digital_exclusion_score', 'risk_level']
   Expected: ['district_id', 'month', 'migration_score', 'migration_category', 'peri_urban_label', 'digital_exclusion_score', 'risk_level']
   ✅ Schema matches exactly

✅ No missing values

📊 Data Types:
   district_id: object
   month: object
   migration_score: float64
   migration_category: object
   peri_urban_label: object
   digital_exclusion_score: int64
   risk_level: object

📄 Sample merged data:


,district_id,month,migration_score,migration_category,peri_urban_label,digital_exclusion_score,risk_level
0,GJ_AMD,2024-01,0.0017,Moderate Inflow,Stable Urban,25,Low
1,GJ_AMD,2024-02,0.0154,High Inflow,Declining,28,Low
2,GJ_AMD,2024-03,0.0105,Moderate Inflow,Declining,31,Low
3,GJ_AMD,2024-04,0.0006,Moderate Inflow,Declining,25,Low
4,GJ_AMD,2024-05,0.0246,High Inflow,Declining,24,Low
5,GJ_AMD,2024-06,0.0202,High Inflow,Declining,25,Low
6,GJ_AMD,2024-07,0.0054,Moderate Inflow,Declining,31,Low
7,GJ_AMD,2024-08,0.0144,Moderate Inflow,Declining,24,Low
8,GJ_AMD,2024-09,0.0211,High Inflow,Declining,30,Low
9,GJ_AMD,2024-10,-0.0023,Moderate Outflow,Declining,25,Low


## 5. Export CSV (snake_case)

In [5]:
"""
Export CSV
----------
Save the merged dataset in CSV format with snake_case column names.
This file will be read by the backend API.
"""

# CSV already uses snake_case column names
dashboard_df.to_csv(CSV_OUTPUT_PATH, index=False)

print(f"✅ CSV exported to: {CSV_OUTPUT_PATH}")
print(f"   Rows: {len(dashboard_df)}")
print(f"   Columns: {list(dashboard_df.columns)}")

✅ CSV exported to: d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\api_ready\aadhaar_pulse_dashboard.csv
   Rows: 240
   Columns: ['district_id', 'month', 'migration_score', 'migration_category', 'peri_urban_label', 'digital_exclusion_score', 'risk_level']


## 6. Export JSON (camelCase)

Convert column names to camelCase for JSON output (frontend convention).

In [6]:
"""
Export JSON (camelCase)
-----------------------
Convert snake_case to camelCase for frontend consumption.
Output is an array of objects.
"""

# =============================================================================
# CAMELCASE COLUMN MAPPING
# =============================================================================
CAMEL_CASE_MAPPING = {
    'district_id': 'districtId',
    'month': 'month',  # No change needed
    'migration_score': 'migrationScore',
    'migration_category': 'migrationCategory',
    'peri_urban_label': 'periUrbanLabel',
    'digital_exclusion_score': 'digitalExclusionScore',
    'risk_level': 'riskLevel'
}

# Create JSON-ready dataframe with camelCase columns
json_df = dashboard_df.rename(columns=CAMEL_CASE_MAPPING)

# Convert to list of dictionaries (array of objects)
json_data = json_df.to_dict(orient='records')

# Export to JSON file
with open(JSON_OUTPUT_PATH, 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=2, ensure_ascii=False)

print(f"✅ JSON exported to: {JSON_OUTPUT_PATH}")
print(f"   Records: {len(json_data)}")
print(f"   Keys: {list(json_data[0].keys()) if json_data else 'N/A'}")
print(f"\n📄 Sample JSON record:")
print(json.dumps(json_data[0], indent=2) if json_data else 'N/A')

✅ JSON exported to: d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\api_ready\aadhaar_pulse_dashboard.json
   Records: 240
   Keys: ['districtId', 'month', 'migrationScore', 'migrationCategory', 'periUrbanLabel', 'digitalExclusionScore', 'riskLevel']

📄 Sample JSON record:
{
  "districtId": "GJ_AMD",
  "month": "2024-01",
  "migrationScore": 0.0017,
  "migrationCategory": "Moderate Inflow",
  "periUrbanLabel": "Stable Urban",
  "digitalExclusionScore": 25,
  "riskLevel": "Low"
}


## 7. Final Summary

Phase 6 complete. Dashboard data is now backend-ready.

In [7]:
"""
Final Summary
-------------
Display summary statistics for the merged dashboard dataset.
"""

print("=" * 60)
print("PHASE 6 COMPLETE: Dashboard Data Merge")
print("=" * 60)

print(f"\n📁 Output Files Created:")
print(f"   1. {CSV_OUTPUT_PATH}")
print(f"   2. {JSON_OUTPUT_PATH}")

print(f"\n📊 Dataset Statistics:")
print(f"   Total records: {len(dashboard_df)}")
print(f"   Districts: {dashboard_df['district_id'].nunique()}")
print(f"   Months: {dashboard_df['month'].nunique()}")

print(f"\n📋 Final Schema (CSV - snake_case):")
for col in FINAL_COLUMNS:
    print(f"   • {col}")

print(f"\n📋 Final Schema (JSON - camelCase):")
for snake, camel in CAMEL_CASE_MAPPING.items():
    print(f"   • {camel}")

print(f"\n✅ Data is ready for backend API and frontend dashboard!")
print("=" * 60)

PHASE 6 COMPLETE: Dashboard Data Merge

📁 Output Files Created:
   1. d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\api_ready\aadhaar_pulse_dashboard.csv
   2. d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\api_ready\aadhaar_pulse_dashboard.json

📊 Dataset Statistics:
   Total records: 240
   Districts: 20
   Months: 12

📋 Final Schema (CSV - snake_case):
   • district_id
   • month
   • migration_score
   • migration_category
   • peri_urban_label
   • digital_exclusion_score
   • risk_level

📋 Final Schema (JSON - camelCase):
   • districtId
   • month
   • migrationScore
   • migrationCategory
   • periUrbanLabel
   • digitalExclusionScore
   • riskLevel

✅ Data is ready for backend API and frontend dashboard!
